In [19]:
import yaml
import pandas as pd 
controls = ['negative_control','positive_control']
grn_models = ['collectri','granie', 'figr', 'celloracle', 'scglue', 'scenicplus']

# Robustness analysis

In [35]:
!viash ns build --setup cb --parallel

temporaryFolder: /tmp/viash_hub_repo15216710281727276246 uri: https://github.com/openproblems-bio/openproblems-v2.git
Cloning into '.'...
checkout out: List(git, checkout, origin/main_build, --, .) 0 
temporaryFolder: /tmp/viash_hub_repo7041524287744599172 uri: https://github.com/openproblems-bio/openproblems-v2.git
Cloning into '.'...
checkout out: List(git, checkout, origin/main_build, --, .) 0 
Exporting positive_control (control_methods) =docker=> /mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_benchmark/target/docker/control_methods/positive_control
Exporting create_component (common) =docker=> /mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_benchmark/target/docker/common/create_component
Exporting batch_correction_seurat (perturbation) =native=> /mnt/c/Users/nourisa/Documents/testProjs/ongoing/task_grn_benchmark/target/native/perturbation/batch_correction_seurat
Exporting normalization (perturbation) =nextflow=> /mnt/c/Users/nourisa/Documents/testProjs/ongoing

## Shuffle net

In [40]:
!bash scripts/run_robust_analys.sh "net"

net
Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [astonishing_poincare] DSL2 - revision: 6bd5cda1f4
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

[-        ] process > run_robustness_analysis:run... [  0%] 0 of 4
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (6)
[4b/08b1c4] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run...

In [42]:
base_folder = 'output/robust_analy_net'

result_file = f'{base_folder}/scores.yaml'
with open(result_file, 'r') as file:
    data = yaml.safe_load(file)
df_reg1 = pd.DataFrame({}, columns=grn_models)
# Iterate over the data
for entry in data:
    method_id = entry['method_id']
    reg_type, degree, method = method_id.split('-')
    if reg_type == 'reg1':
        col = 'ex(True)_tf(-1)'
        ex_true_tf_idx = entry['metric_ids'].index(col)
        ex_true_tf_value = entry['metric_values'][ex_true_tf_idx]
        df_reg1.loc[int(degree), method] = ex_true_tf_value

df_reg1.sort_index()

,collectri,granie,figr,celloracle,scglue,scenicplus
0,-0.224471,0.185105,0.186789,0.228927,0.234921,0.33327
10,-0.315255,0.170255,0.139809,0.203669,0.132937,0.312483
20,-0.369273,0.160628,0.125023,0.183348,0.156769,0.293282
50,-0.421345,0.105859,0.067059,0.110983,0.081232,0.231999
100,-0.456039,-0.028658,-0.090232,-0.003224,-0.10102,-0.018023


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(5,3), sharey=True)
df_reg1.index = df_reg1.index.map(surragate_names)
df_reg1.columns = df_reg1.columns.map(surragate_names)

ax = axes[0]
headmap(df_reg1, name='', ax=ax, fmt='.03f')
ax.xaxis.set_label_position('top')
ax.xaxis.tick_top()
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='left')

# ax = axes[1]
# scores_rank = scores_mat.rank(ascending=False)
# scores_rank['Mean'] = scores_rank.mean(axis=1)
# headmap_rank(scores_rank, name='', ax=ax)
# ax.xaxis.set_label_position('top')
# ax.xaxis.tick_top()
# ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='left')

# plt.savefig(f'{results_folder}/benchmark/figs/final-reg2.png', dpi=300, transparent=True,  bbox_inches='tight')

## Add noise to weights

In [41]:
!bash scripts/run_robust_analys.sh "weight"

weight
Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [dreamy_davinci] DSL2 - revision: 6bd5cda1f4
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (1)
[5f/aa4675] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (6)
[f3/689356] process > run_robustness_analysis:run... [  0%] 0 of 30
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustn

In [43]:
base_folder = 'output/robust_analy_weight'

result_file = f'{base_folder}/scores.yaml'
with open(result_file, 'r') as file:
    data = yaml.safe_load(file)
df_reg1 = pd.DataFrame({}, columns=grn_models)
# Iterate over the data
for entry in data:
    method_id = entry['method_id']
    reg_type, degree, method = method_id.split('-')
    if reg_type == 'reg1':
        col = 'ex(True)_tf(-1)'
        ex_true_tf_idx = entry['metric_ids'].index(col)
        ex_true_tf_value = entry['metric_values'][ex_true_tf_idx]
        df_reg1.loc[int(degree), method] = ex_true_tf_value

df_reg1.sort_index()

,collectri,granie,figr,celloracle,scglue,scenicplus
0,-0.224471,0.185105,0.186789,0.228927,0.234921,0.33327
10,-0.224716,0.185143,0.18504,0.227575,0.225391,0.333008
20,-0.228165,0.183382,0.180036,0.224755,0.221201,0.332981
50,-0.235282,0.174765,0.164224,0.20491,0.167678,0.325849
100,-0.297882,0.146087,0.117188,0.167833,0.120502,0.304062


## Negative vs positive controls

In [36]:
!bash scripts/run_pc_vs_nc.sh "pc_vs_nc"

Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [magical_mercator] DSL2 - revision: 6bd5cda1f4
[-        ] process > run_robustness_analysis:run... -

[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (2)
[c6/b7f5d5] process > run_robustness_analysis:run... [  0%] 0 of 18
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:pub... -

executor >  local (6)
[4f/bc6528] process > run_robustness_analysis:run... [  0%] 0 of 18
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_analysis:run... -
[-        ] process > run_robustness_a

In [39]:
base_folder = 'output/robust_analy_pc_vs_nc'

result_file = f'{base_folder}/scores.yaml'
with open(result_file, 'r') as file:
    data = yaml.safe_load(file)
df_reg1 = pd.DataFrame({}, columns=grn_models)
# Iterate over the data
for entry in data:
    method_id = entry['method_id']
    reg_type, rest = method_id.split('--')
    subsample, method = rest.split('-')
    if reg_type == 'reg1':
        col = 'ex(True)_tf(-1)'
        ex_true_tf_idx = entry['metric_ids'].index(col)
        ex_true_tf_value = entry['metric_values'][ex_true_tf_idx]
        df_reg1.loc[int(subsample), method] = ex_true_tf_value

df_reg1 = df_reg1.sort_index()
df_reg1.index = ['Default', 'Negative control', 'Positive control']
df_reg1

,collectri,granie,figr,celloracle,scglue,scenicplus
Default,-0.224471,0.185105,0.186789,0.228927,0.234921,0.33327
Negative control,-0.249304,0.200482,0.211147,0.234089,0.250881,0.358298
Positive control,-0.242072,0.175922,0.179734,0.215799,0.218924,0.316465


## regulatory sign

In [46]:
!bash scripts/run_robust_analys.sh "sign"

sign
Nextflow 24.04.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.4
Launching `target/nextflow/workflows/run_robustness_analysis/main.nf` [cranky_perlman] DSL2 - revision: 6bd5cda1f4
